In [3]:
from biogeme.database import Database
import pandas as pd

swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
database = Database("swissmetro", swissmetro)
globals().update(database.variables)

# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database.remove(exclude)
database.data["CHOICE"] -= 1


In [553]:
import numpy as np
import aesara
import aesara.tensor as aet
from biogeme.database import Database
import pandas as pd
from biogeme_aesara import DatabaseShared, BetaShared


swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
db = DatabaseShared("swissmetro", swissmetro, choiceVar="CHOICE")
globals().update(db.variables)


# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
db.remove(exclude)
db.data["CHOICE"] -= 1


b_cost = BetaShared("b_cost", 0., None, None, 0)
b_time = BetaShared("b_time", 0., None, None, 0)
asc_train = BetaShared("asc_train", 0., None, None, 0)
asc_car = BetaShared("asc_car", 0., None, None, 0)
asc_sm = BetaShared("asc_sm", 0., None, None, 0)

print(asc_sm.status)

# load all BetaShared Python variables into params
params = []
params_full = []
for _, val in locals().items():
    if isinstance(val, BetaShared):
        params_full.append(val())
        if val.status == 0:
            params.append(val())
print(b_cost)
print(b_cost + 1)
print(type(asc_train))

U_1 = b_cost * TRAIN_CO.x + b_time * TRAIN_TT.x + asc_train
U_2 = b_cost * CAR_CO.x + b_time * CAR_TT.x + asc_car
U_3 = b_cost * SM_CO.x + b_time * SM_TT.x + asc_sm

print("U_1", type(U_1))

print(db.data[[TRAIN_CO.name]].shape)
print(b_cost.sharedVar.eval().shape)
Uvec = aet.concatenate([U_1, U_2, U_3], axis=1)
prob = aet.nnet.softmax(Uvec)

loglike = aet.mean(aet.log(prob)[aet.arange(CHOICE.y.shape[0]), CHOICE.y])
grads = aet.grad(loglike, params)

print("b_cost_shape", b_cost.sharedVar.shape.eval())

# hess, updates = aesara.scan(
# 	lambda i, y, x: aet.grad(
# 		y[i],
# 		x,
# 		consider_constant=None,
# 		disconnected_inputs="ignore",
# 	),
# 	sequences=range(len(grads)),
# 	non_sequences=[grads, params],
# )

hess = aet.as_tensor_variable(np.zeros((len(grads), len(grads))))
for i in range(len(grads)):
	hess = aet.set_subtensor(hess[i,:], aet.grad(grads[i], params,
		consider_constant=None,
		disconnected_inputs="ignore"))

bhhh = aet.outer(aet.as_tensor_variable(grads), aet.as_tensor_variable(grads).T)

double1 = aet.grad(grads[0], params)

# double2 = aet.grad(g_b_time, [b_time(), b_cost()])


updates = [(param, param + 0.0001 * grad) for param, grad in zip(params, grads)]
f = aesara.function(
    db.get_x() + db.get_y(),
    hess,
    updates=updates,
    on_unused_input="ignore",
)

bh = aesara.function(
    db.get_x() + db.get_y(),
    bhhh,
    on_unused_input="ignore",
)

estimated_betas = aesara.function(
    db.get_x() + db.get_y(),
    params_full,
    on_unused_input="ignore",
) 

output = f(*(db.get_x_data() + db.get_y_data()))
print(output)
test_bh = bh(*(db.get_x_data() + db.get_y_data()))
print(test_bh)
test_betas = estimated_betas(*(db.get_x_data() + db.get_y_data()))
print(np.asarray(test_betas))

0
b_cost(0.0)
(b_cost(0.0) + `1`)
<class 'biogeme_aesara.BetaShared'>
U_1 <class 'aesara.tensor.var.TensorVariable'>
(6768, 1)
()
b_cost_shape []
[[-431180.90625      -4284.74072266     -29.11644363     108.29333496
      -79.17687988]
 [  -4284.74072266   -2877.74316406     -13.8325119        0.47501436
       13.35750103]
 [    -29.11644745     -13.83251095      -0.22222222       0.11111111
        0.11111111]
 [    108.29332733       0.47501278       0.11111111      -0.22222222
        0.11111111]
 [    -79.17687988      13.35749626       0.11111111       0.11111111
       -0.22222222]]
[[44902.617        601.8054       -32.45117       27.600248
      4.850923  ]
 [  601.8054         8.065671      -0.43492544     0.36991113
      0.06501429]
 [  -32.45117       -0.43492544     0.0234525     -0.01994673
     -0.00350577]
 [   27.600248       0.36991113    -0.01994673     0.01696502
      0.00298171]
 [    4.850923       0.06501429    -0.00350577     0.00298171
      0.00052406]]
[-0.

In [609]:
from scipy import linalg
output = f(*(db.get_x_data() + db.get_y_data()))
np.set_printoptions(suppress=True)
print("hessian matrix:")
print(output)
varcovar = -linalg.pinv(np.nan_to_num(output))
stdErr = []
for i in range(len(params)):
    if varcovar[i, i] < 0:
        stdErr.append(np.finfo(float).max)
    else:
        stdErr.append(np.sqrt(varcovar[i, i]))
print("standard errors:")
print(stdErr)

d = np.diag(varcovar)
if (d > 0).all():
    diag = np.diag(np.sqrt(d))
    diagInv = linalg.inv(diag)
    correlation = diagInv.dot(varcovar.dot(diagInv))
else:
    correlation = np.full_like(varcovar, np.finfo(float).max)

print("correlation matrix:")
print(correlation)

test_bh = bh(*(db.get_x_data() + db.get_y_data()))
# test_bh = linalg.inv(np.nan_to_num(test_bh))
# print("BHHH:")
# print(test_bh)
robust_varCovar = varcovar.dot(test_bh.dot(varcovar))
robStdErr = []
for i in range(len(params)):
    if robust_varCovar[i, i] < 0:
        robStdErr.append(np.finfo(float).max)
    else:
        robStdErr.append(np.sqrt(robust_varCovar[i, i]))
rd = np.diag(robust_varCovar)
if (rd > 0).all():
    diag = np.diag(np.sqrt(rd))
    diagInv = linalg.inv(diag)
    robust_correlation = diagInv.dot(robust_varCovar.dot(diagInv))
else:
    robust_correlation = np.full_like(robust_varCovar, np.finfo(float).max)

print("rob correlation matrix:")
print(robust_correlation)   

test_betas = estimated_betas(*(db.get_x_data() + db.get_y_data()))
print(np.asarray(test_betas))

hessian matrix:
[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
standard errors:
[-0.0, -0.0, -0.0, -0.0, -0.0]
correlation matrix:
[[1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]]
rob correlation matrix:
[[1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308
  1.79769313e+308]
 [1.797693